<a href="https://colab.research.google.com/github/khushee-g/EEG/blob/main/Copy_of_Muse_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np # Linear algebra
import pandas as pd
# ModelLing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics


In [41]:
#Kaggle - baseline for healthy control

dataKaggle = pd.read_csv('/content/numericBehaviorWithAgeOnly.csv')
dataKaggle = dataKaggle[dataKaggle['specific_disorder'] == 4]

column_averages = dataKaggle.mean()

for column in dataKaggle.columns:
    dataKaggle[column] = column_averages[column]

dataKaggle = dataKaggle.iloc[:1]
dataKaggle


,age,specific_disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,AB.A.delta.g.F8,AB.A.delta.h.T3,...,COH.A.delta.i.C3.p.P4,COH.A.delta.i.C3.q.T6,COH.A.delta.i.C3.r.O1,COH.A.delta.i.C3.s.O2,COH.A.delta.j.Cz.k.C4,COH.A.delta.j.Cz.l.T4,COH.A.delta.j.Cz.m.T5,COH.A.delta.j.Cz.n.P3,COH.A.delta.j.Cz.o.Pz,COH.A.delta.j.Cz.p.P4
0,25.724737,4.0,18.383673,19.49709,15.526428,17.398592,18.687952,17.443004,16.333668,9.459207,...,45.517665,27.983055,29.402465,25.619368,77.705019,39.334421,35.28447,57.108869,68.867802,57.224611


In [42]:
#Muse - finding average power for each channel
dataMuse = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

dataMuse = dataMuse.iloc[0:, 1:21]
column_averages = dataMuse.mean()

for column in dataMuse.columns:
    dataMuse[column] = column_averages[column]

dataMuse = dataMuse.iloc[:1]
dataMuse

<ipython-input-42-3c45a5090862>:2: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  dataMuse = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,Alpha_AF7,Alpha_AF8,Alpha_TP10,Beta_TP9,Beta_AF7,Beta_AF8,Beta_TP10,Gamma_TP9,Gamma_AF7,Gamma_AF8,Gamma_TP10
0,0.511999,0.321635,0.490609,0.576975,0.364001,0.15997,0.152903,0.425637,0.742932,0.499081,0.302163,0.753088,0.54797,1.190189,0.034327,0.535749,0.24495,1.001965,-0.350563,0.213928


In [43]:
#Delta AF8-TP9 coherence - corresponds to COH.A.delta.g.F8.i.C3

from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = data['Delta_AF8'].values
channel_2 = data['Delta_TP9'].values

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF8DeltaTP9': Cxy})

coherence_AF8_TP9 = coherence_data['Coherence DeltaAF8DeltaTP9'].mean()
print(coherence_AF8_TP9)


0.1391618213308134


<ipython-input-43-fb46db4ec331>:5: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [44]:
#Delta AF7-AF8 coherence - corresponds to COH.A.delta.a.FP1.b.FP2

from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = data['Delta_AF7'].values
channel_2 = data['Delta_AF8'].values

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Delta AF7-AF8 coherence': Cxy})

coherence_AF8_AF7 = coherence_data['Delta AF7-AF8 coherence'].mean()
print(coherence_AF8_AF7)

0.13365575864956636


<ipython-input-44-83044c5ff5f6>:5: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [1]:
risk = 0

F7F3 = (dataMuse['Delta_AF7']-(dataKaggle['COH.A.delta.c.F7.d.F3']/100))*0.590723 #0.590723 is the odds ratio from Complete Processing Colab File .ipynb
if F7F3.iloc[0] < 0:
  risk += F7F3.iloc[0]
if F7F3.iloc[0] > 0:
  risk -= F7F3.iloc[0]


FP1FP2 = (coherence_AF8_AF7-(dataKaggle['COH.A.delta.a.FP1.b.FP2']/100))*0.00173307
if FP1FP2.iloc[0] < 0:
  risk -= FP1FP2.iloc[0]
if FP1FP2.iloc[0] > 0:
  risk += FP1FP2.iloc[0]


F8C3 = (coherence_AF8_TP9-(dataKaggle['COH.A.delta.g.F8.i.C3']/100))*0.610099
if F8C3.iloc[0] < 0:
  risk -= F8C3.iloc[0]
if F8C3.iloc[0] > 0:
  risk += F8C3.iloc[0]


print(risk)

if risk>0:
  print("You are "+str(risk *100)+" times more at risk to develop behavioral addiction")
if risk<0:
  print("You are "+str(risk*-100)+" times less at risk to develop behavioral addiction")

NameError: name 'dataMuse' is not defined